# DB 化

フィールドの説明は [ここ](https://open.fda.gov/apis/drug/label/searchable-fields/) から見られる

In [1]:
import pandas as pd
import sqlite3
import os

In [13]:
csv_file = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_columns.csv"  # カラム情報
tsv_file = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/drug-label-0001-of-0012.tsv" # data

columns_info = pd.read_csv(csv_file)
tsv_data = pd.read_csv(tsv_file, sep='\t', low_memory=False)

In [3]:
# 除外しないカラムを選択
included_columns = columns_info[(columns_info['除外する'] == 'FALSE') | (columns_info['除外する'] == '△')]['DLしたデータの列'].values
exist_columns = [col for col in included_columns if col in tsv_data.columns]

# data から除外しないカラムを抽出
tsv_data_filtered = tsv_data[exist_columns]

In [4]:
# CSVにあるがTSVにない列を追加（欠損値で埋める）
tsv_data_filtered = tsv_data_filtered.copy()

missing_columns = set(included_columns) - set(tsv_data_filtered.columns)
for col in missing_columns:
    tsv_data_filtered[col] = None

In [5]:
# 列の順番をCSVに基づいて整理
tsv_data_filtered = tsv_data_filtered[included_columns]

In [6]:
tsv_data_filtered

,set_id,id,abuse,accessories,active_ingredient,adverse_reactions,alarms,animal_pharmacology_and_or_toxicology,ask_doctor,ask_doctor_or_pharmacist,...,summary_of_safety_and_effectiveness,teratogenic_effects,troubleshooting,use_in_specific_populations,user_safety_warnings,version,veterinary_indications,warnings,warnings_and_cautions,when_using
0,4d5bfd3a-3793-4d24-aedd-c04fa34ea736,1fb1cd4e-2efd-4893-885a-c05994325955,NaN,NaN,NaN,['ADVERSE REACTIONS The following adverse reac...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6,NaN,"['WARNINGS Addiction, Abuse, and Misuse Hydroc...",NaN,NaN
1,ef3590cc-49ae-44f7-85a9-7c11f58d467e,8ce3fe4e-f32a-4028-a920-86d638628cca,"['Abuse Dextroamphetamine saccharate, amphetam...",NaN,NaN,['ADVERSE REACTIONS Cardiovascular Palpitation...,None,NaN,NaN,NaN,...,NaN,"['Teratogenic Effects Amphetamine, in the enan...",NaN,NaN,NaN,7,NaN,"[""WARNINGS Abuse, Misuse, and Addiction Dextro...",NaN,NaN
2,392a846e-7325-462d-b19b-8afda44c5cb0,c41e5c06-bede-4ccc-b5d5-ebed1867077c,['ABUSE Hydrocodone bitartrate and acetaminoph...,NaN,NaN,['ADVERSE REACTIONS The following adverse reac...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6,NaN,"['WARNINGS Addiction, Abuse, and Misuse Hydroc...",NaN,NaN
3,55af77dd-2abc-49a6-990c-399d5a5e7a91,dbf69168-7068-4db3-834e-c33980303fd9,NaN,NaN,NaN,['6 ADVERSE REACTIONS The following adverse re...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,"[""8 USE IN SPECIFIC POPULATIONS Infertility: N...",NaN,2,NaN,NaN,['5 WARNINGS AND PRECAUTIONS Hepatotoxicity: I...,NaN
4,4951ec29-609b-4836-b0e4-0d9c1d6ae6fe,036c6990-7539-f75f-e063-6294a90a7cd2,"['9.2 Abuse Buprenorphine, like morphine and o...",NaN,NaN,['6 ADVERSE REACTIONS The following serious ad...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,['8 USE IN SPECIFIC POPULATIONS Lactation : Bu...,NaN,2,NaN,NaN,['5 WARNINGS AND PRECAUTIONS Contains sodium m...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,36c5c818-0d40-445e-a3c0-0f46723a25e2,22617d3b-9736-472b-bc08-1ce53fe151aa,NaN,NaN,NaN,['6 ADVERSE REACTIONS The most common adverse ...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,"[""8 USE IN SPECIFIC POPULATIONS 8.1 Pregnancy ...",NaN,6,NaN,NaN,"[""5 WARNINGS AND PRECAUTIONS Hypercalcemia and...",NaN
19996,11c3e74a-5d0a-4e7f-be87-48345b2692e5,53a6a8ed-d8c3-4ece-8346-93953a4e2c56,NaN,NaN,['Active ingredient (in each caplet) Diphenhyd...,NaN,None,NaN,['Ask a doctor before use if you have a breath...,['Ask a doctor or pharmacist before use if you...,...,NaN,NaN,NaN,NaN,NaN,8,NaN,['Warnings Do not use in children under 12 yea...,NaN,['When using this product use caution when dri...
19997,6605ced6-68fe-4d7d-baa6-2526f7c9a64a,661f0649-d87e-46cc-9ad0-ffc478a95d58,['9.2 Abuse Morphine Sulfate Oral Solution con...,NaN,NaN,['6 ADVERSE REACTIONS The following serious ad...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,['8 USE IN SPECIFIC POPULATIONS Pregnancy: May...,NaN,2,NaN,NaN,['5 WARNINGS AND PRECAUTIONS • Life-Threatenin...,NaN
19998,75418d32-6504-4cb4-b3a9-afa0eaf19b8d,89460e99-bf96-4d67-8182-b3cfbab1227b,"['9.2 Abuse Abuse is the intentional, non-ther...",NaN,NaN,['6 ADVERSE REACTIONS The following serious ad...,None,NaN,NaN,NaN,...,NaN,NaN,NaN,"[""8 USE IN SPECIFIC POPULATIONS Pregnancy: Bas...",NaN,100,NaN,NaN,['5 WARNINGS AND PRECAUTIONS Drug Reaction wit...,NaN


20000 行で 600 MB とかっぽい。4000 行ごとに区切る。

In [14]:
chunk_size = 4000  # 4000行ごとに分割
num_rows = len(tsv_data_filtered)
num_chunks = (num_rows // chunk_size) + 1
base_db_file = "fdalabel"  # DBファイル名
table_name = "raw"

In [15]:
chunk_start = 0
chunk_index = 1

In [37]:
while chunk_start < num_rows:
    db_file = f"/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_db/{base_db_file}_{chunk_index:04d}.db"
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # テーブルを作るテーブルを作る
    columns_sql = ", ".join([f'"{col}" TEXT' for col in tsv_data_filtered.columns])
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"
    cursor.execute(create_table_sql)

    # 分割
    chunk_end = min(chunk_start + chunk_size, num_rows)
    chunk = tsv_data_filtered.iloc[chunk_start:chunk_end]

    column_names = ", ".join([f'"{col}"' for col in chunk.columns])
    placeholders = ", ".join(["?" for _ in chunk.columns])
    insert_sql = f'INSERT INTO "{table_name}" ({column_names}) VALUES ({placeholders})'
    
    cursor.executemany(insert_sql, chunk.values.tolist())
    conn.commit()

    chunk_start = chunk_end
    chunk_index += 1

    conn.close()

In [36]:
print(insert_sql)

INSERT INTO 'raw' (set_id, id, abuse, accessories, active_ingredient, adverse_reactions, alarms, animal_pharmacology_and_or_toxicology, ask_doctor, ask_doctor_or_pharmacist, assembly_or_installation_instructions, boxed_warning, calibration_instructions, carcinogenesis_and_mutagenesis_and_impairment_of_fertility, cleaning, clinical_pharmacology, clinical_studies, compatible_accessories, components, contraindications, controlled_substance, dependence, description, diagram_of_device, disposal_and_waste_handling, do_not_use, dosage_and_administration, dosage_forms_and_strengths, drug_abuse_and_dependence, drug_and_or_laboratory_test_interactions, drug_interactions, effective_time, environmental_warning, food_safety_warning, general_precautions, geriatric_use, guaranteed_analysis_of_feed, health_care_provider_letter, health_claim, how_supplied, inactive_ingredient, indications_and_usage, information_for_owners_or_caregivers, information_for_patients, instructions_for_use, intended_use_of_th

# フォルダ内のファイル全部をまとめて実行

In [3]:
os.listdir(tsv_dir)

['drug-label-0004-of-0012.tsv',
 'drug-label-0001-of-0012.tsv',
 'drug-label-0006-of-0012.tsv',
 'drug-label-0005-of-0012.tsv',
 'drug-label-0010-of-0012.tsv',
 'drug-label-0009-of-0012.tsv',
 'drug-label-0002-of-0012.tsv',
 'drug-label-0011-of-0012.tsv',
 'drug-label-0012-of-0012.tsv',
 'drug-label-0007-of-0012.tsv',
 'drug-label-0003-of-0012.tsv',
 'drug-label-0008-of-0012.tsv']

In [6]:
import pandas as pd
import sqlite3
import os

csv_file = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_columns.csv"  # カラム情報
columns_info = pd.read_csv(csv_file)

# TSVファイルが格納されているディレクトリ
tsv_dir = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/"

chunk_index = 1

tsv_files = [f for f in os.listdir(tsv_dir) if f.endswith(".tsv")]
tsv_files.sort()

# ディレクトリ内のすべてのTSVファイルを処理
for tsv_file in tsv_files:
    if tsv_file.endswith(".tsv"):
        tsv_file_path = os.path.join(tsv_dir, tsv_file)
        
        tsv_data = pd.read_csv(tsv_file_path, sep='\t', low_memory=False)

        # 除外しないカラムを選択
        included_columns = columns_info[(columns_info['除外する'] == 'FALSE') | (columns_info['除外する'] == '△')]['DLしたデータの列'].values
        exist_columns = [col for col in included_columns if col in tsv_data.columns]

        # data から除外しないカラムを抽出
        tsv_data_filtered = tsv_data[exist_columns]

        # CSVにあるがTSVにない列を追加（欠損値で埋める）
        tsv_data_filtered = tsv_data_filtered.copy()
        missing_columns = set(included_columns) - set(tsv_data_filtered.columns)
        for col in missing_columns:
            tsv_data_filtered[col] = None

        # 列の順番をCSVに基づいて整理
        tsv_data_filtered = tsv_data_filtered[included_columns]

        chunk_size = 4000  # 4000行ごとに分割
        num_rows = len(tsv_data_filtered)
        num_chunks = (num_rows // chunk_size) + 1
        base_db_file = "fdalabel"  # DBファイル名
        table_name = "raw"

        chunk_start = 0

        while chunk_start < num_rows:
            db_file = f"/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_db/{base_db_file}_{chunk_index:04d}.db"
            conn = sqlite3.connect(db_file)
            cursor = conn.cursor()

            # テーブルを作る
            columns_sql = ", ".join([f'"{col}" TEXT' for col in tsv_data_filtered.columns])
            create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"
            cursor.execute(create_table_sql)

            # 分割
            chunk_end = min(chunk_start + chunk_size, num_rows)
            chunk = tsv_data_filtered.iloc[chunk_start:chunk_end]

            column_names = ", ".join([f'"{col}"' for col in chunk.columns])
            placeholders = ", ".join(["?" for _ in chunk.columns])
            insert_sql = f'INSERT INTO "{table_name}" ({column_names}) VALUES ({placeholders})'

            cursor.executemany(insert_sql, chunk.values.tolist())
            conn.commit()

            chunk_start = chunk_end
            chunk_index += 1

            conn.close()